In [1]:
import pandas as pd

df = pd.read_csv('data/merged_df.csv')
df

,ids,autobiography,bibliography,biography,case-control_studies,case_reports,clinical_conference,clinical_studies_as_topic,clinical_study,clinical_trial,...,veterinary_clinical_trial,veterinary_randomized_controlled_trial,ids_str,PMCID,article_id,article_text,abstract_text,labels,section_names,sections
0,19934601,0,0,0,0,0,0,0,1,1,...,1,1,19934601,PMC2807272,PMC2807272,['a hernia is a protrusion of the contents of ...,['<S> umbilical hernias in calves commonly pre...,NaN,"['Introduction', 'Materials and Methods', 'Ani...",[['a hernia is a protrusion of the contents of...
1,25641234,0,0,0,0,0,0,0,1,1,...,1,1,25641234,PMC4895487,PMC4895487,['calves were separated from their dam immedia...,['<S> objectiveto evaluate the effect of orall...,NaN,"['Material and Methods', 'Animals', 'Experimen...",[['the study was carried out on 24 male calves...
2,26725871,0,0,0,0,0,0,0,1,1,...,1,1,26725871,PMC4873845,PMC4873845,['43/04 ) by the \n local ethics committee on ...,['<S> the aim of the study was to investigate ...,NaN,"['MATERIALS AND METHODS', 'RESULTS', 'DISCUSSI...",[['43/04 ) by the \n local ethics committee on...
3,27481572,0,0,0,0,0,0,0,1,1,...,1,1,27481572,PMC5032872,PMC5032872,"['specifically , their presenting problems inc...",['<S> objectivespublished descriptions of the ...,NaN,"['Materials and Methods', 'Horses', 'Oral Suga...",[['the study population consisted of 10 adult ...
4,25720808,0,0,0,0,0,0,0,1,1,...,1,1,25720808,PMC4349540,PMC4349540,['animals : the experiment was approved by the...,['<S> the present study evaluated \n the effec...,NaN,"['MATERIALS AND METHODS', 'RESULTS', 'DISCUSSI...",[['animals : the experiment was approved by th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,25069512,0,0,0,0,0,0,0,0,0,...,0,0,25069512,PMC4262937,PMC4262937,['micrornas ( mirnas ) are small non - coding ...,['<S> micrornas post - transcriptionally regul...,NaN,"['INTRODUCTION', 'MiRNAs IN STEM CELLS', 'MiRN...",[['micrornas ( mirnas ) are small non - coding...
4176,26823664,0,0,0,0,0,0,0,0,0,...,0,0,26823664,PMC4707359,PMC4707359,['living beings are sensitive to magnetic fiel...,['<S> it is known that the geomagnetic field c...,NaN,"['1. Introduction', '2. Geomagnetic Field', '3...",[['living beings are sensitive to magnetic fie...
4177,23703336,0,0,0,0,0,0,0,0,0,...,0,0,23703336,PMC3724249,PMC3724249,['diabetes is a major cause of health concern ...,['<S> pancreas plays an important role in main...,NaN,"['Introduction', 'Dysfunctional pancreas in di...",[['diabetes is a major cause of health concern...
4178,24189465,0,0,0,0,0,0,0,0,0,...,0,0,24189465,PMC3829700,PMC3829700,['the definition and operationalization of emp...,"['<S> from a multidimensional perspective , em...",NaN,"['Empathy and Narcissism', 'Neural Mechanisms ...",[['the definition and operationalization of em...


In [2]:
import ast

df['article_text'] = df['article_text'].apply(ast.literal_eval)
type(df['article_text'][0])

list

In [3]:
import pandas as pd

cols = df.columns.drop(['ids', 'ids_str', 'PMCID', 'article_id', 'article_text', 'abstract_text', 'labels',	'section_names', 'sections'])

# 把每一列的那幾個欄位變成 list（或 numpy array）
df['label_vector'] = df[cols].values.tolist()

# 查看結果
print(df['label_vector'][0])


[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['article_text'].tolist(), df['label_vector'].tolist(), test_size=0.1, random_state=42, )


In [12]:
len(y_train[0])

61

In [15]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer("neuml/pubmedbert-base-embeddings")



In [9]:
from tqdm import tqdm
import numpy as np
train_vectors = []

for sentences in tqdm(X_train):
    embeddings = model.encode(sentences)
    avg_vector = np.mean(embeddings, axis=0)
    train_vectors.append(avg_vector)

train_vectors = np.array(train_vectors)


100%|██████████| 3762/3762 [09:46<00:00,  6.41it/s]


TypeError: 'tuple' object is not callable

In [16]:
from tqdm import tqdm
import numpy as np
test_vectors = []

for sentences in tqdm(X_test):
    embeddings = model.encode(sentences)
    avg_vector = np.mean(embeddings, axis=0)
    test_vectors.append(avg_vector)

test_vectors = np.array(test_vectors)


100%|██████████| 418/418 [01:05<00:00,  6.37it/s]


In [11]:
train_vectors.shape

(3762, 768)

In [12]:
np.save('middle_files/train_vectors.npy', train_vectors)


In [17]:
np.save('middle_files/test_vectors.npy', test_vectors)


In [26]:
np.save('middle_files/y_train.npy', np.array(y_train))
np.save('middle_files/y_test.npy', np.array(y_test))

In [21]:
train_vectors

array([[ 0.6092773 , -0.0918682 ,  0.41433764, ...,  0.00847781,
         0.4390122 ,  0.7156233 ],
       [ 0.3526911 ,  0.10651661,  0.0116943 , ...,  0.10747708,
         0.3058618 , -0.06458779],
       [ 0.31382743,  0.4664917 , -0.10677963, ..., -0.20160058,
         0.28766432, -0.37294996],
       ...,
       [-0.09152881, -0.2706452 ,  0.3406826 , ..., -0.06567764,
        -0.40885615,  0.10527508],
       [-0.04398777,  0.12514417,  0.51953757, ..., -0.01990971,
        -0.08445261,  0.19840214],
       [ 0.22140796,  0.30133703,  0.02396924, ..., -0.04940783,
         0.12327175,  0.05115751]], dtype=float32)

## load

In [27]:
import numpy as np
train_vectors = np.load('middle_files/train_vectors.npy')
test_vectors =  np.load('middle_files/test_vectors.npy')
y_train = np.load('middle_files/y_train.npy')
y_test = np.load('middle_files/y_test.npy')

In [28]:
import torch
import torch.nn as nn

class ClassifierHead(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256, output_dim=len(y_train[0])):
        super(ClassifierHead, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()  # for multi-label classification
        )

    def forward(self, x):
        return self.classifier(x)


In [29]:
model = ClassifierHead(output_dim=len(y_train[0]))
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

X_tensor = torch.tensor(train_vectors, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)

for epoch in range(100):
    model.train()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")


Epoch 1 Loss: 0.6922
Epoch 2 Loss: 0.6909
Epoch 3 Loss: 0.6896
Epoch 4 Loss: 0.6883
Epoch 5 Loss: 0.6870
Epoch 6 Loss: 0.6857
Epoch 7 Loss: 0.6843
Epoch 8 Loss: 0.6830
Epoch 9 Loss: 0.6815
Epoch 10 Loss: 0.6801
Epoch 11 Loss: 0.6785
Epoch 12 Loss: 0.6769
Epoch 13 Loss: 0.6751
Epoch 14 Loss: 0.6733
Epoch 15 Loss: 0.6714
Epoch 16 Loss: 0.6693
Epoch 17 Loss: 0.6671
Epoch 18 Loss: 0.6647
Epoch 19 Loss: 0.6622
Epoch 20 Loss: 0.6595
Epoch 21 Loss: 0.6567
Epoch 22 Loss: 0.6536
Epoch 23 Loss: 0.6504
Epoch 24 Loss: 0.6469
Epoch 25 Loss: 0.6432
Epoch 26 Loss: 0.6393
Epoch 27 Loss: 0.6351
Epoch 28 Loss: 0.6306
Epoch 29 Loss: 0.6259
Epoch 30 Loss: 0.6208
Epoch 31 Loss: 0.6154
Epoch 32 Loss: 0.6097
Epoch 33 Loss: 0.6036
Epoch 34 Loss: 0.5972
Epoch 35 Loss: 0.5904
Epoch 36 Loss: 0.5832
Epoch 37 Loss: 0.5757
Epoch 38 Loss: 0.5678
Epoch 39 Loss: 0.5594
Epoch 40 Loss: 0.5507
Epoch 41 Loss: 0.5415
Epoch 42 Loss: 0.5320
Epoch 43 Loss: 0.5220
Epoch 44 Loss: 0.5117
Epoch 45 Loss: 0.5010
Epoch 46 Loss: 0.48

In [30]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, hamming_loss, classification_report

def evaluate_model(model, X, y_true, threshold=0.5):
    """
    Evaluate a multi-label classifier.
    
    Args:
        model: Trained PyTorch model.
        X: torch.Tensor of shape (N, input_dim).
        y_true: numpy array of shape (N, num_labels).
        threshold: Classification threshold for sigmoid outputs.
    """
    model.eval()
    with torch.no_grad():
        y_probs = model(X)
        y_pred = (y_probs > threshold).int().cpu().numpy()
        y_true = y_true.cpu().numpy() if hasattr(y_true, 'cpu') else y_true

    print("📊 Evaluation Results:")
    print(f"Hamming Loss:        {hamming_loss(y_true, y_pred):.4f}")
    print(f"Micro F1-score:      {f1_score(y_true, y_pred, average='micro'):.4f}")
    print(f"Macro F1-score:      {f1_score(y_true, y_pred, average='macro'):.4f}")
    print(f"Weighted F1-score:   {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Subset Accuracy:     {accuracy_score(y_true, y_pred):.4f}")
    
    print("\n🧾 Classification Report:")
    print(classification_report(y_true, y_pred))


In [31]:
X_test_tensor = torch.tensor(test_vectors, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

evaluate_model(model, X_test_tensor, y_test_tensor)


📊 Evaluation Results:
Hamming Loss:        0.0276
Micro F1-score:      0.4971
Macro F1-score:      0.0292
Weighted F1-score:   0.3818
Subset Accuracy:     0.2392

🧾 Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         6
           7       0.73      0.97      0.83       234
           8       0.97      0.18      0.31       179
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         4
 

/opt/packages/anaconda3-2024.10-1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/packages/anaconda3-2024.10-1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/packages/anaconda3-2024.10-1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri